In [1]:
from astropy.io import fits
import numpy as np
from tensorflow import keras
import pickle
from numpy import exp
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
fits_file = "../mastDownload/TESS/tess2021175071901-s0040-0000000027769688-0211-a_fast/tess2021175071901-s0040-0000000027769688-0211-a_fast-lc.fits"
fits.info(fits_file)
fits.getdata(fits_file, ext=1).columns

with fits.open(fits_file, mode="readonly") as hdulist:
    pdcsap_fluxes = hdulist[1].data['PDCSAP_FLUX']

Filename: ../mastDownload/TESS/tess2021175071901-s0040-0000000027769688-0211-a_fast/tess2021175071901-s0040-0000000027769688-0211-a_fast-lc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      44   ()      
  1  LIGHTCURVE    1 BinTableHDU    167   121861R x 20C   [D, E, J, E, E, E, E, E, E, J, D, E, D, E, D, E, D, E, E, E]   
  2  APERTURE      1 ImageHDU        49   (11, 11)   int32   


In [3]:
def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    data = (data - mean) / std
    return data

In [4]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [5]:
#retrieve and format data - into labels and examples from the dataset
def features_and_labels(filename):
    data = np.matrix(pd.read_csv(filename).values)

    #we need to transpose data sets to be compatable with our weight vectors
    data_y = data[:, 0]
    data_y-=1 #binaryization of the categorical data

    data_x = data[:, 1:]
    data_x = convert_3D(normalize(data_x))
    return data_x, data_y

Data imputation for nan's in flux data: predict data or delete nans

In [6]:
# trim data to fit matrix
pdcsap_fluxes = pdcsap_fluxes[np.logical_not(np.isnan(pdcsap_fluxes))]
low = (np.floor((len(pdcsap_fluxes)/3197))*3197).astype(int)
pdcsap_fluxes = pdcsap_fluxes[0:low]
pdcsap_fluxes = normalize(pdcsap_fluxes)
pdcsap_fluxes = np.reshape(pdcsap_fluxes, (-1,3197))
pdcsap_fluxes_3D = np.expand_dims(pdcsap_fluxes, -1)

In [7]:
test_x, test_y = features_and_labels("../archive/exoTest.csv")
test_x = test_x[1,:]
test_x = np.expand_dims(test_x, -1)

In [8]:
#function that outputs true if ANY of the probabilities are greater than .50, else false
def predict(probs):
    for num in probs:
        if num > 0.50:
            return True
    return False

In [9]:
# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("keras-models/KerasLSTM")
#print(reconstructed_model.predict(pdcsap_fluxes_3D))
#print("(LSTM) Is there an exoplanet? ", predict(reconstructed_model.predict(pdcsap_fluxes_3D)))

print(reconstructed_model.predict(test_x))
print("(LSTM) Is there an exoplanet? ", predict(reconstructed_model.predict(test_x)))

[[0.4974519 ]
 [0.4974443 ]
 [0.49745142]
 ...
 [0.49719384]
 [0.49719244]
 [0.49720958]]
(LSTM) Is there an exoplanet?  False


In [10]:
# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("keras-models/KerasGRU")

print("(GRU) Is there an exoplanet? ", predict(reconstructed_model.predict(pdcsap_fluxes_3D)))

(GRU) Is there an exoplanet?  False


In [11]:
# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("keras-models/KerasRNN")

print("(RNN) Is there an exoplanet? ", predict(reconstructed_model.predict(pdcsap_fluxes_3D)))

(RNN) Is there an exoplanet?  False


In [12]:
#Logistic Layer using a sigmoid function
def logistic_layer(y):
    y = np.array(y)
    y = 1 / (1 + exp(-y)) # sigmoid function
    y = y.ravel()
    return y

In [13]:
with open('tf-models/multi-lr-dft.npy', 'rb') as f:
    W = np.load(f)
    b = np.load(f)
    
predicted_y = W * pdcsap_fluxes + b
predicted_y = logistic_layer(predicted_y)

print("(LR-DFT) Is there an exoplanet? ", predict(predicted_y))

(LR-DFT) Is there an exoplanet?  True


In [14]:
with open('tf-models/multi-lr-ols.npy', 'rb') as f:
    W = np.load(f)
    
predicted_y = W * pdcsap_fluxes + b
predicted_y = logistic_layer(predicted_y)

print("(LR-OLS) Is there an exoplanet? ", predict(predicted_y))

(LR-OLS) Is there an exoplanet?  True


In [15]:
with open('tf-models/multi-lr.npy', 'rb') as f:
    W = np.load(f)
    b = np.load(f)
    
predicted_y = W * pdcsap_fluxes + b
predicted_y = logistic_layer(predicted_y)

print("(LR) Is there an exoplanet? ", predict(predicted_y))

(LR) Is there an exoplanet?  True


In [16]:
with open('tf-models/decision-tree.pkl', 'rb') as f:
    dt = pickle.load(f)

pred = True
if 0 in dt.predict(pdcsap_fluxes):
    pred = False

print("(Decision Tree) Is there an exoplanet? ", pred)

(Decision Tree) Is there an exoplanet?  False


In [17]:
with open('tf-models/random-forest.pkl', 'rb') as f:
    rf = pickle.load(f)

pred = True
if 0 in rf.predict(pdcsap_fluxes):
    pred = False

    
print("(Random Forest) Is there an exoplanet? ", pred)

(Random Forest) Is there an exoplanet?  False
